In [1]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #


import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm

import xtrack as xt
import xobjects as xo
import xpart as xp


beta_rel = 0.305
#beta_rel = 0.106

gamma = 1.050
#gamma = 1.006


current=2.4
length = 1.5 # m cooler length
r_beam=25*1e-3

mass0=938.27208816*1e6 #ev/c^2

T_perp = 100e-3 # <E> [eV] = kb*T
T_l =  1e-3 # <E> [eV]
magnetic_field = 0.060 # T for AD
B_ratio=1e-10



c=299792458.0

p0c = mass0*beta_rel*gamma #eV/c



circumference = 182.43280000000 #m
T = circumference/(c*beta_rel)
s_per_turn = T


beta_x=10 
beta_y=4

#disp_x=0.12

                                        


Parameters: $I=2.4 A, B=0.060 T, \beta_x=10 m,\beta_y=4,D_x=0.12 m T_\perp=100meV,T_\parallel=1meV,\epsilon_{x}=35*1e-6,\epsilon_{y}=35*1e-6,dp/p=1e-3$

In [2]:
import numpy as np

delta_single = 5e-4
num_turns = int(1e4)

# Define the objective function to minimize
def objective_function(params):
    disp_x, offset_x, offset_dp = params
    
    particles = xp.Particles(
        mass0=mass0,
        p0c=p0c,
        x=disp_x * delta_single,
        px=0,
        y=0,
        py=0,
        delta=delta_single + offset_dp,
        zeta=0
    )

    arc = xt.LinearTransferMatrix(
        Q_x=5.44, Q_y=5.42,
        beta_x_0=beta_x, beta_x_1=beta_x,
        beta_y_0=beta_y, beta_y_1=beta_y,
        alpha_x_0=0, alpha_x_1=0,
        alpha_y_0=0, alpha_y_1=0,
        disp_x_0=disp_x, disp_x_1=disp_x,
        disp_y_0=0, disp_y_1=0,
        beta_s=1 * 1e40,
        Q_s=0,
        chroma_x=0.0, chroma_y=0
    )

    cooler = xt.ElectronCooler(
        current=current,
        length=length,
        r_beam=r_beam,
        T_perp=T_perp,
        T_l=T_l,
        magnetic_field=magnetic_field,
        B_ratio=1e-4,
        Neutralisation_space_charge=1,
        offset_dp=offset_dp,
        offset_x=offset_x
    )

    action_x = []
    delta = []

    for turn in range(num_turns):
        action_x_temp = (particles.x ** 2 / beta_x + beta_x * particles.px ** 2)
        action_x.append(action_x_temp)
        delta.append(particles.delta)

        arc.track(particles)
        cooler.track(particles)

    action_reduction = action_x[-1] 
    delta_reduction = delta[-1] / delta[0]

    # Minimize the action reduction
    return action_reduction


def perform_grid_search(parameter_ranges):
    best_action_reduction = np.inf
    best_params = None

    for disp_x in parameter_ranges['disp_x']:
        for offset_x in parameter_ranges['offset_x']:
            for offset_dp in parameter_ranges['offset_dp']:
                params = [disp_x, offset_x, offset_dp]
                action_reduction = objective_function(params)
            
                if action_reduction < best_action_reduction:
                    best_action_reduction = action_reduction
                    best_params = params

    return best_params, best_action_reduction


# Define the parameter ranges for the grid search
parameter_ranges = {
    'disp_x': np.linspace(-20, 20, num=20),
    'offset_x': np.linspace(-5e-4, 5e-4, num=20),
    'offset_dp': np.linspace(-1e-9, 1e-9, num=40)
}

# Perform the grid search
best_params, best_action_reduction = perform_grid_search(parameter_ranges)

# Get the optimal values and action reduction
optimal_disp_x, optimal_offset_x, optimal_offset_dp = best_params

print("Optimal values:")
print("disp_x:", optimal_disp_x)
print("Offset_x:", optimal_offset_x)
print("Offset_dp:", optimal_offset_dp)
print("Best Action Reduction:", best_action_reduction)


Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Optimal values:
disp_x: 1.0526315789473664
Offset_x: -0.0005
Offset_dp: 1e-09
Best Action Reduction: [2.47109707e-08]
